In [ ]:
import matplotlib.pyplot as plt
import machi.base as ac
import numpy as np
from hypso.download import download_nc_files

In [ ]:
download_nc_files(filename_list=["bergen_2024-06-11_1048Z-l1a.nc"],
                  download_dir="../../Downloads")

In [ ]:
from hypso import Hypso

hypso_file_path="../../Downloads/bergen_2024-06-11_1048Z-l1a.nc"
# load and calibrate the hypso image, e.g. convert to radiance
satobj = Hypso(hypso_file_path)

In [ ]:
wl = satobj.wavelengths[4:118]

In [ ]:
cube = satobj.get_toa_reflectance()[:,:,4:118]

In [ ]:
plt.imshow(cube[:,:,100])
plt.colorbar()

In [ ]:
sat = np.max(satobj.rawcube, axis=-1) > 35000

In [ ]:
plt.imshow(sat)
plt.colorbar()

In [ ]:
T, A, objs = ac.atm_correction(cube.reshape(-1,114), solar=np.ones(114), verbose=True, tol=0.1)

In [ ]:
fig, ax = plt.subplots(1)
ax.plot(wl,T, color='black')
ax.set_ylabel("transmission")
ax0 = ax.twinx()
ax0.set_ylabel("back scatter (fraction of incident light)", color='blue')
ax0.plot(wl,A, color='blue')

In [ ]:
cube_norm = (cube - A) /T
cube_norm[sat] = np.nan

In [ ]:
plt.plot(cube_norm[400,600])
plt.plot(cube[400,600])

In [ ]:
plt.imshow(cube_norm[:,:,20], vmax=.01)

In [ ]:
rgb = np.zeros(cube_norm.shape[:2] + (3,))

In [ ]:
rgb[:,:,0] = cube_norm[:,:,65]
rgb[:,:,1] = cube_norm[:,:,47]
rgb[:,:,2] = cube_norm[:,:,20]
rgb*=20
rgb[rgb>1] =1
rgb[np.isnan(rgb)]=1

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
plt.imshow(np.rot90(rgb), aspect=0.3)